In [ ]:
# Import libraries
from sklearn.model_selection import train_test_split
import keras
from keras.layers import Dense
from keras.models import Model, Sequential

import pandas as pd

import tensorflow as tf

import os
from keras.layers import Dropout
import mat

print(tf.version.VERSION)

In [ ]:
INPUT_DATA_DIR = "preprocessed_input/"
TRAIN_DATA_COEFFICIENT = 0.70

files = []

for (dirpath, dirnames, filenames) in os.walk(INPUT_DATA_DIR):
    files.extend(filenames)
    break

In [ ]:
INPUT_DATA_DIR = "preprocessed_input/"


def tf_data_generator(file_list, batch_size = 1):
    counter = 0

    while True:
        fname = file_list[counter]

        counter = (counter + 1) % len(file_list)
        df = pd.read_csv(INPUT_DATA_DIR + fname)
        df.dtypes
        
        
        df.drop(['pickup_datetime'], axis=1, errors='ignore')
        
        input = df[['pickup_longitude','dropoff_longitude','pickup_latitude',
                  'dropoff_latitude','passenger_count','year','weekday','weekend','hour_sin','month_sin',
                  'delta_longitude','delta_latitude','distance_km','direction',
                  'dropoff_manhattan','pickup_manhattan','peak_hours','JFK','LGA',
                  'NWK']].copy().astype(float)
        
        
        output = df['fare_amount'].copy()


        for local_index in range(0, input.shape[0], batch_size):
            input_local = input[local_index:(local_index + batch_size)]
            output_local = output[local_index:(local_index + batch_size)]
            print(output_local.shape)

            yield input_local, output_local

In [ ]:
train_file_names, test_file_names = train_test_split(files, test_size = 0.2, random_state = 321)

train_file_names, validation_file_names = train_test_split(train_file_names, test_size = 0.15, random_state = 232)


In [ ]:
print("Number of train_files:" ,len(train_file_names))
print("Number of validation_files:" ,len(validation_file_names))
print("Number of test_files:" ,len(test_file_names))

In [2]:
batch_size = 1000000
train_dataset = tf.data.Dataset.from_generator(
                                             generator=lambda: tf_data_generator(file_list=train_file_names, batch_size=batch_size), 
                                              output_shapes = ((None, 20), (None,)),
                                              output_types = (tf.float32, tf.float32))

validation_dataset = tf.data.Dataset.from_generator(generator=lambda: tf_data_generator(file_list=validation_file_names, batch_size=batch_size), 
                                                      output_shapes = ((None, 20), (None,)),
                                                   output_types = (tf.float32, tf.float32))

test_dataset = tf.data.Dataset.from_generator(generator=lambda: tf_data_generator(file_list=test_file_names, batch_size=batch_size), 
                                              output_shapes = ((None, 20), (None,)),
                                             output_types = (tf.float32, tf.float32))


In [3]:
def build_model():
  model = Sequential()
  model.add(Dense(20, activation='relu', input_dim=20))
  model.add(Dropout(0.2))
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(32, activation='relu')) 
  model.add(Dropout(0.2))
  model.add(Dense(16, activation='relu'))
  model.add(Dense(1,  activation='relu'))

  model.compile(loss='mse', optimizer = 'adam', metrics=['mse','mae',"accuracy"])
  return model

# create model
model = build_model()
model.summary()

In [4]:
# callback_list = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=1)]

hist = model.fit(
    steps_per_epoch=len(train_file_names),
    use_multiprocessing=True,
    workers=6,
    x=train_dataset,
    verbose=1,
    max_queue_size=32,
    epochs=100,
#     callbacks=callback_list,
    validation_data=test_dataset,
    validation_steps=len(validation_file_names)
)

In [ ]:
model.save('new_model')

print("Test loss: ", test_loss)
print("Test accuracy:", test_accuracy)